In [5]:
import cv2
import numpy as np
import moviepy.editor as mp
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Function to extract frames from the video with frame downsampling
def extract_frames(video_clip, frame_rate, sample_rate=1):
    frames = []
    for i, frame in enumerate(video_clip.iter_frames(fps=frame_rate, dtype="uint8")):
        if i % sample_rate == 0:  # Only take every `sample_rate`-th frame
            frames.append(frame)
    return frames

# Function to extract features from each frame (using color histograms)
def extract_frame_features(frames):
    features = []
    for frame in frames:
        # Convert frame to grayscale for simplicity
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        # Compute the histogram of the frame
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten()  # Flatten the histogram into a 1D array
        features.append(hist)
    return np.array(features)

# Function to select key frames (one per cluster)
def select_key_frames(frames, labels, num_clusters):
    key_frames = []
    for i in range(num_clusters):
        # Get all frames that belong to the current cluster
        cluster_frames = [frames[j] for j in range(len(frames)) if labels[j] == i]
        # Select the first frame of the cluster (this can be optimized further)
        key_frames.append(cluster_frames[0])
    return key_frames

# Load video file using MoviePy
video_path = "C:/Users/91982/Desktop/sample1.mp4"  # Updated with your video path
video_clip = mp.VideoFileClip(video_path)

# Get video details
frame_rate = video_clip.fps  # Frame rate of the video
video_duration = video_clip.duration  # Duration of the video in seconds

# Check if the video is too short (less than 20 seconds)
if video_duration < 20:
    # For very short videos, just keep the original video as the summary
    summary_video = video_clip.subclip(0, video_duration)
    print("Video is too short. Returning the original video as the summary.")
else:
    # Reduce frame size (e.g., every 5th frame) to save memory
    sample_rate = 5  # Only take every 5th frame
    frames = extract_frames(video_clip, frame_rate, sample_rate)
    print(f"Extracted {len(frames)} frames.")

    # Extract features from frames (using histograms)
    frame_features = extract_frame_features(frames)
    print(f"Extracted features for {len(frames)} frames.")

    # Apply KMeans clustering to detect scenes
    num_clusters = 5  # Number of key moments/scenes
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(frame_features)

    # Print the clustering labels for each frame
    print(f"Clustering labels: {labels}")

    # Select key frames from each cluster
    key_frames = select_key_frames(frames, labels, num_clusters)
    print(f"Selected {len(key_frames)} key frames.")

    # Convert key frames to moviepy clips
    key_frame_clips = [mp.ImageClip(frame).set_duration(2).set_fps(frame_rate) for frame in key_frames]

    # Concatenate clips into a single summary video
    summary_video = mp.concatenate_videoclips(key_frame_clips, method="compose")

# Save the summary video
summary_video.write_videofile("summary_video.mp4", codec="libx264")

# Display the summary video in the notebook (if using Jupyter)
from IPython.display import Video
Video("summary_video.mp4")


Extracted 307 frames.
Extracted features for 307 frames.
Clustering labels: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4]
Selected 5 key frames.
Moviepy - Building video summary_video.mp4.
Moviepy - Writing video summary_video.mp4



Moviepy - Done !
Moviepy - video ready summary_video.mp4
